<a href="https://colab.research.google.com/github/FizaKonnur06/ResumeAnalyzerProject0.2/blob/main/AI_Resume_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk scikit-learn PyPDF2 python-docx pandas


In [ ]:
import nltk
import string
import pandas as pd
import numpy as np
import io # Added import for BytesIO

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import PyPDF2
import docx

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab') # Added to download the missing resource

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:

from google.colab import files

uploaded = files.upload()


Saving Fiza A Konnur.pdf to Fiza A Konnur.pdf


In [ ]:
def extract_text(file_name, file_content):
    text = ""

    # Determine file type based on file_name extension
    if file_name.lower().endswith(".pdf"):
        reader = PyPDF2.PdfReader(io.BytesIO(file_content)) # Use BytesIO for content
        for page in reader.pages:
            extracted_page_text = page.extract_text()
            if extracted_page_text: # Ensure text was actually extracted
                text += extracted_page_text + " " # Add space for readability
            else:
                # Handle cases where extract_text might return None or empty string
                # e.g., scanned PDFs without selectable text
                print(f"Warning: No text extracted from a page in {file_name}. It might be an image-based PDF.")

    elif file_name.lower().endswith(".docx"):
        doc = docx.Document(io.BytesIO(file_content)) # Use BytesIO for content
        for para in doc.paragraphs:
            text += para.text + " "

    return text

resume_file_name = list(uploaded.keys())[0]
resume_file_content = uploaded[resume_file_name] # Get the byte content
resume_text = extract_text(resume_file_name, resume_file_content) # Pass both

print(resume_text[:1000])   # preview

FIZA ABIDALI KONNUR
CAREER OBJECTIVE
Detail-oriented and motivated IT graduate seeking an entry-level IT Support / Service
Desk role where I can assist end users with application, system, and network-related
issues, utilize my technical and communication skills, and contribute effectively to
organizational success while continuously learning and growing in the IT domain.8073998152
EDUCATION
SOFT SKILLShttps://github.com/FizaKonnur06
Bachelor of Computer Application(BCA)-Rani Channamma University ,Belguam|2023-
Present|CGPA:8.9
PUC(Commerce)-Bapuji Gramin Vikas Samithi College, Ramnagar|2021-
2023|Percentage:83.66%
SSLC-Bashiban Urdu High School,Khanapur|2018-2021|Percentage:88.64%
TECHNICAL SKILLS
 Programming Launguages:C,Python,java,C#(Basics) 
Web Technologies: HTML,CSS,Javascript(Basics)
Database Management:MYSql  
Tools and Platforms:Github,visual Studio,PowerBI
Other Concepts: Data structure using C,OOPs,Operating System,Computer
Communication Networks,Software engineering.
Excel

### Job Description

Here, we define the job description against which resumes will be compared. This description can be updated to match different job roles and requirements.

In [ ]:
job_description = """
We are looking for a Python developer with knowledge of
Machine Learning, NLP, SQL, Flask, and AWS.
Experience in data analysis is a plus.
"""

print("Job Description:\n", job_description)

Job Description:
 
We are looking for a Python developer with knowledge of
Machine Learning, NLP, SQL, Flask, and AWS.
Experience in data analysis is a plus.



In [ ]:
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))

    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stopwords.words('english')]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w) for w in tokens]

    return " ".join(tokens)

clean_resume = clean_text(resume_text)
clean_job = clean_text(job_description)


In [ ]:
skill_list = [
    "python", "java", "machine learning", "deep learning",
    "sql", "nlp", "aws", "docker", "flask",
    "data analysis", "pandas", "numpy"
]


In [ ]:
def extract_skills(text, skills):
    extracted = []
    for skill in skills:
        if skill in text:
            extracted.append(skill)
    return list(set(extracted))

resume_skills = extract_skills(clean_resume, skill_list)
job_skills = extract_skills(clean_job, skill_list)

print("Resume Skills:", resume_skills)
print("Job Required Skills:", job_skills)


Resume Skills: ['pandas', 'machine learning', 'python', 'sql', 'java', 'data analysis', 'numpy']
Job Required Skills: ['nlp', 'machine learning', 'aws', 'python', 'sql', 'data analysis', 'flask']


In [ ]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([clean_resume, clean_job])

similarity_score = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
match_percentage = round(similarity_score * 100, 2)

print("Match Percentage:", match_percentage, "%")


Match Percentage: 10.11 %


In [ ]:
matched_skills = list(set(resume_skills) & set(job_skills))
missing_skills = list(set(job_skills) - set(resume_skills))

print("Matched Skills:", matched_skills)
print("Missing Skills:", missing_skills)


Matched Skills: ['machine learning', 'sql', 'data analysis', 'python']
Missing Skills: ['nlp', 'flask', 'aws']


In [ ]:
total_job_skills = len(job_skills)
matched_count = len(matched_skills)

match_percentage = (matched_count / total_job_skills) * 100

print("Match Percentage:", round(match_percentage,2), "%")


Match Percentage: 57.14 %


In [ ]:
missing_count = len(missing_skills)
skill_gap = (missing_count / total_job_skills) * 100

print("Skill Gap:", round(skill_gap,2), "%")


Skill Gap: 42.86 %


In [ ]:
if match_percentage >= 75:
    level = "🔥 Highly Suitable"
elif match_percentage >= 50:
    level = "👍 Moderately Suitable"
else:
    level = "⚠️ Low Match – Needs Improvement"

print("Candidate Level:", level)


Candidate Level: 👍 Moderately Suitable


In [ ]:
print("\n----- AI Career Recommendation -----")

if match_percentage >= 75:
    print("Excellent! Your resume strongly matches this job role.")
    print("You can confidently apply for this position.")

elif match_percentage >= 50:
    print("Good match! But improving the missing skills will increase your chances.")
    print("Focus on learning:", ", ".join(missing_skills))

else:
    print("Your resume does not match well with this job.")
    print("You should learn these skills before applying:")
    print(", ".join(missing_skills))



----- AI Career Recommendation -----
Good match! But improving the missing skills will increase your chances.
Focus on learning: nlp, flask, aws


In [ ]:
print("\n----- RESUME ANALYSIS -----")

total_resume_skills = len(resume_skills)
print("Total Skills Found:", total_resume_skills)
print("Skills Detected:", resume_skills)



----- RESUME ANALYSIS -----
Total Skills Found: 7
Skills Detected: ['pandas', 'machine learning', 'python', 'sql', 'java', 'data analysis', 'numpy']


In [ ]:
data_roles = ["python","machine learning","data analysis","statistics","pandas","numpy","sql","nlp","deep learning"]
web_roles = ["html","css","javascript","react","node","mongodb","express"]
cloud_roles = ["aws","azure","gcp","docker","kubernetes"]

domain = "General"

if any(skill in resume_skills for skill in data_roles):
    domain = "Data Science / Machine Learning"

elif any(skill in resume_skills for skill in web_roles):
    domain = "Web Development"

elif any(skill in resume_skills for skill in cloud_roles):
    domain = "Cloud / DevOps"

print("Detected Resume Domain:", domain)


Detected Resume Domain: Data Science / Machine Learning


In [ ]:
data_roles = ["python","machine learning","data analysis","statistics","pandas","numpy","sql","nlp","deep learning"]
web_roles = ["html","css","javascript","react","node","mongodb","express"]
cloud_roles = ["aws","azure","gcp","docker","kubernetes"]

domain = "General"

if any(skill in resume_skills for skill in data_roles):
    domain = "Data Science / Machine Learning"

elif any(skill in resume_skills for skill in web_roles):
    domain = "Web Development"

elif any(skill in resume_skills for skill in cloud_roles):
    domain = "Cloud / DevOps"

print("Detected Resume Domain:", domain)


Detected Resume Domain: Data Science / Machine Learning


In [ ]:
if total_resume_skills >= 10:
    experience = "Experienced / Job Ready"
elif total_resume_skills >= 6:
    experience = "Intermediate"
else:
    experience = "Beginner / Fresher"

print("Experience Level:", experience)


Experience Level: Intermediate


In [ ]:
resume_strength = min(total_resume_skills * 10, 100)
print("Resume Strength Score:", resume_strength, "/100")


Resume Strength Score: 70 /100


In [ ]:
print("\n----- Resume Summary -----")

summary = f"""
This resume belongs to a {experience} candidate.
The profile is mainly focused on {domain}.
The resume shows {resume_strength}% strength based on skills detected.
"""

print(summary)



----- Resume Summary -----

This resume belongs to a Intermediate candidate.
The profile is mainly focused on Data Science / Machine Learning.
The resume shows 70% strength based on skills detected.



In [ ]:
print("\n📊 AI Resume Analysis Report")
print("---------------------------")
print(f"✔ Match Score: {match_percentage}%")
print(f"🟢 Matched Skills: {matched_skills}")
print(f"🔴 Missing Skills: {missing_skills}")



📊 AI Resume Analysis Report
---------------------------
✔ Match Score: 57.14285714285714%
🟢 Matched Skills: ['machine learning', 'sql', 'data analysis', 'python']
🔴 Missing Skills: ['nlp', 'flask', 'aws']


In [ ]:
!pip install streamlit pyngrok nltk scikit-learn PyPDF2 python-docx


%%writefile app.py
import streamlit as st
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import PyPDF2
import docx

# Download NLTK data (safe in Streamlit)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# ---------- FUNCTIONS ----------

def extract_text(uploaded_file):
    text = ""
    if uploaded_file.name.endswith(".pdf"):
        reader = PyPDF2.PdfReader(uploaded_file)
        for page in reader.pages:
            text += page.extract_text()
    elif uploaded_file.name.endswith(".docx"):
        doc = docx.Document(uploaded_file)
        for para in doc.paragraphs:
            text += para.text + " "
    return text


def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    return " ".join(tokens)


def extract_skills(text):
    skills = [
        "python","java","machine learning","deep learning",
        "sql","aws","flask","pandas"
    ]
    return [s for s in skills if s in text]

# ---------- STREAMLIT UI ----------

st.title("AI Resume Analyzer & Job Matcher")

uploaded_file = st.file_uploader("Upload Resume (PDF or DOCX)", type=["pdf","docx"])
job_desc = st.text_area("Paste Job Description")

if st.button("Analyze"):
    if uploaded_file and job_desc:
        resume_text = extract_text(uploaded_file)
        clean_resume = clean_text(resume_text)
        clean_job = clean_text(job_desc)

        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([clean_resume, clean_job])
        score = cosine_similarity(vectors[0:1], vectors[1:2])[0][0] * 100

        resume_skills = extract_skills(clean_resume)
        job_skills = extract_skills(clean_job)

        matched = list(set(resume_skills) & set(job_skills))
        missing = list(set(job_skills) - set(resume_skills))

        st.success(f"Match Score: {round(score,2)}%")
        st.write("Matched Skills:", matched)
        st.write("Missing Skills:", missing)
    else:
        st.warning("Please upload a resume and paste job description.")


In [ ]:
!streamlit run app.py &>/content/logs.txt &


In [ ]:
!ngrok config add-authtoken 39CBEJaCBoRY7BHQ0XwrvtxMjEE_3XbHfbMomkkEwQSFdJd1t


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print("Your app is live at:", public_url)


Your app is live at: NgrokTunnel: "https://personable-margo-euphonically.ngrok-free.dev" -> "http://localhost:8501"


# Task
Analyze the resume `FIZA ABIDALI KONNUR.pdf` against the provided job description to identify relevant skills, calculate a match percentage, and generate a report summarizing matched and missing skills.

## Project Overview

### Subtask:
Provide an introductory note explaining the purpose of this project, which is to analyze and match a resume against a given job description.


## Project Overview

### Subtask:
Provide an introductory note explaining the purpose of this project, which is to analyze and match a resume against a given job description.

### Introduction
This project aims to analyze a provided resume against a specific job description. It identifies relevant skills present in the resume, compares them with the skills required by the job description, and calculates a match percentage to indicate the resume's suitability for the role. This tool can help job seekers tailor their resumes and recruiters efficiently screen candidates.

## File Upload and Text Extraction

### Subtask:
Detail how the resume file was uploaded and how text was extracted from it using PyPDF2, ensuring it can handle various document formats.


### File Upload and Text Extraction

To begin, the resume file was uploaded to the Colab environment using `google.colab.files.upload()`. This function provides a convenient way for users to upload files directly from their local system into the Colab runtime, making them accessible for processing.

Following the upload, a custom function named `extract_text` was implemented to handle text extraction from various document formats. This function is designed to intelligently parse both PDF and DOCX files:

- **For PDF files**: It utilizes the `PyPDF2.PdfReader` library. The uploaded file content, which is in bytes, is first wrapped with `io.BytesIO` to allow `PyPDF2` to read it as a file-like object. The function then iterates through each page of the PDF, using `page.extract_text()` to retrieve the textual content. A warning is printed if a page yields no text, which can happen with image-based PDFs.

- **For DOCX files**: It uses the `docx.Document` library. Similar to PDF handling, `io.BytesIO` is employed to process the byte content of the DOCX file. The function then iterates through all paragraphs in the document, concatenating their text content.

The `io.BytesIO` object plays a crucial role by providing an in-memory binary stream, making the uploaded file content compatible with both `PyPDF2` and `docx` libraries, which expect file-like objects as input.

Finally, the extracted text from the uploaded resume is stored in the `resume_text` variable. A preview of the first 1000 characters of `resume_text` was then printed to verify that the text extraction was successful and to get a glimpse of the content.

## Job Description Definition

### Subtask:
Explain how the job description was defined and why it's a crucial input for the matching process.


### Job Description Definition

#### How the `job_description` was defined:

The `job_description` variable was explicitly defined in the notebook as a multi-line string. This allows for a detailed and human-readable representation of the desired job role and its requirements. For instance, in the previous code, it was set as:

```python
job_description = """
We are looking for a Python developer with knowledge of
Machine Learning, NLP, SQL, Flask, and AWS.
Experience in data analysis is a plus.
"""
```

#### Why it's a crucial input for the matching process:

The `job_description` is a critical input because it serves as the **benchmark** or **reference point** against which each resume's suitability is evaluated. The entire matching process, including skill extraction and similarity calculation, hinges on understanding what the job requires. Without a clear and comprehensive job description, the system would not know what skills to look for or how to assess the relevance of the resume content. It directly dictates the 'ideal candidate profile', allowing the algorithm to quantify how well a resume aligns with those specific needs.

## Text Preprocessing

### Subtask:
Describe the text cleaning and normalization steps applied to both the resume and job description, including lowercasing, punctuation removal, tokenization, stop words removal, and lemmatization.


### Text Preprocessing Description

Text preprocessing is a crucial step in preparing raw text data for analysis. Its purpose is to transform unstructured text into a clean and standardized format, which helps in improving the accuracy and efficiency of subsequent analytical tasks, such as skill extraction and similarity calculation. By cleaning and normalizing the text, we reduce noise, handle variations in word forms, and focus on the most meaningful terms.

Our `clean_text` function performs the following steps:

1.  **Lowercasing**: All text is converted to lowercase (`text.lower()`). This ensures that words like "Python", "python", and "PYTHON" are treated as the same word, preventing the system from considering them distinct entities.

2.  **Punctuation Removal**: Punctuation marks (e.g., periods, commas, exclamation marks) are removed using `text.translate(str.maketrans("", "", string.punctuation))`. Punctuation typically doesn't contribute to the meaning of keywords or skills and can introduce unnecessary complexity.

3.  **Tokenization**: The cleaned text is broken down into individual words or "tokens" using `nltk.word_tokenize(text)`. This creates a list of words that can be processed independently.

4.  **Stop Words Removal**: Common English stop words (e.g., 'the', 'is', 'and', 'a') are filtered out (`[w for w in tokens if w not in stopwords.words('english')]`). These words are generally high-frequency but carry little semantic value for identifying specific skills or topics, so removing them helps to focus on more significant terms.

5.  **Lemmatization**: A `WordNetLemmatizer` is applied to reduce words to their base or dictionary form (`[lemmatizer.lemmatize(w) for w in tokens]`). For example, "running," "ran," and "runs" are all reduced to "run." This helps in consolidating different inflections of a word into a single canonical form, improving the consistency of our analysis.

This comprehensive `clean_text` function was applied to both the original `resume_text` to generate `clean_resume` and to the `job_description` to produce `clean_job`, ensuring a standardized and clean input for further processing.

## Skill Extraction

### Subtask:
Outline the process of identifying and extracting relevant skills from both the preprocessed resume and job description based on a predefined skill list.


## Skill Extraction

### Subtask:
Outline the process of identifying and extracting relevant skills from both the preprocessed resume and job description based on a predefined skill list.

#### Explanation of Skill Extraction Process:

1.  **Predefined Skill List**: A `skill_list` (e.g., `"python", "sql", "machine learning"`) serves as a comprehensive reference of relevant technical skills. This list is manually curated to include skills pertinent to the roles being matched.

2.  **`extract_skills` Function**: The core of the skill extraction process is the `extract_skills` function. This function takes two main arguments:
    *   `text`: The preprocessed and cleaned text (either from the resume or the job description).
    *   `skills`: The `skill_list` defined above.

3.  **Iteration and Identification**: Inside the `extract_skills` function, it iterates through each skill in the `skill_list`. For every skill, it checks if that skill string is present within the input `text`. This check is case-insensitive due to the prior cleaning step where all text was converted to lowercase.

4.  **Unique Skill Compilation**: If a skill from the `skill_list` is found in the input `text`, it is added to a temporary list. To avoid duplicate entries, the function ensures that only unique skills are returned by converting the list to a set and then back to a list.

5.  **Application to Resume and Job Description**: This `extract_skills` function is then applied separately to both the `clean_resume` and `clean_job` texts:
    *   `resume_skills = extract_skills(clean_resume, skill_list)`: This call identifies all skills from the predefined `skill_list` that are present in the candidate's resume.
    *   `job_skills = extract_skills(clean_job, skill_list)`: This call identifies all skills from the predefined `skill_list` that are mentioned in the job description.

6.  **Verification**: Finally, printing the `resume_skills` and `job_skills` lists allows for immediate verification of the extracted skills from both sources. This step confirms that the process correctly identified the relevant skills based on the `skill_list` and provides a clear overview of what each document contains in terms of technical competencies.

## Similarity Calculation

### Subtask:
Explain the methodology used to calculate the similarity score between the resume and job description using TF-IDF vectorization and cosine similarity.


```markdown
## Methodology for Similarity Calculation

To determine the similarity between the resume and the job description, a two-step process involving TF-IDF vectorization and cosine similarity is employed.

### 1. TF-IDF Vectorization

**Purpose:** TF-IDF (Term Frequency-Inverse Document Frequency) vectorization converts text documents into numerical representations, which are essential for computational analysis. It weighs words based on their frequency in a document and their rarity across all documents, giving more importance to words that are unique and relevant to a specific document.

**Implementation:**
- The `TfidfVectorizer` from `sklearn.feature_extraction.text` is initialized. This object is responsible for transforming text into a matrix of TF-IDF features.
- The `fit_transform` method of the `TfidfVectorizer` is then applied to a list containing both the `clean_resume` and `clean_job` texts. This step performs two main functions:
    - **`fit`**: It learns the vocabulary and inverse document frequency (IDF) weights from the provided texts.
    - **`transform`**: It converts the texts into numerical vectors. Each document (resume and job description) becomes a vector, where each dimension corresponds to a word in the learned vocabulary, and its value represents the TF-IDF score of that word in the document. A higher score indicates greater importance of that word within the document relative to the entire corpus (in this case, the resume and job description together).

### 2. Cosine Similarity Calculation

**Purpose:** Cosine similarity is a metric used to measure how similar two non-zero vectors are. It measures the cosine of the angle between two vectors. A cosine similarity of 1 means the vectors are identical (same direction), 0 means they are orthogonal (no similarity), and -1 means they are opposite.

**Implementation:**
- The `cosine_similarity` function from `sklearn.metrics.pairwise` is used to calculate the similarity between the two vectorized documents.
- Specifically, it compares the vector of the cleaned resume (`vectors[0:1]`) with the vector of the cleaned job description (`vectors[1:2]`). The output is a matrix, and we extract the single similarity score from it.

**Match Percentage:**
- The resulting `similarity_score` (a value between 0 and 1) is then multiplied by 100 to convert it into a percentage.
- This percentage is rounded to two decimal places to provide a clear and readable `match_percentage`.
- Finally, this `match_percentage` is printed to the console, indicating how well the resume aligns with the job description based on the textual content and skill relevance.
```

## Resume Analysis Report

### Subtask:
Summarize the final output, including the overall match percentage, the skills identified in both documents, and the skills missing from the resume based on the job description requirements.


## Resume Analysis Report

This report culminates the resume analysis process, providing a concise overview of the candidate's suitability for the specified job description. It helps in quickly assessing the relevance of a resume by quantifying its alignment with the job requirements.

### Report Components:
*   **Match Score**: This percentage represents the overall textual and skill alignment between the resume and the job description. A higher score indicates a stronger match in terms of content and keywords.
*   **Matched Skills**: This list highlights the specific skills that were explicitly identified in both the candidate's resume and the job description. These are the direct qualifications the candidate possesses that align with the job's needs.
*   **Missing Skills**: This list enumerates the skills required by the job description that were not found in the candidate's resume. These indicate potential gaps in the candidate's qualifications or areas where the resume could be further tailored to highlight relevant experience.

This report is a valuable tool for both job seekers, who can use it to refine their resumes, and recruiters, who can efficiently screen candidates based on key requirements.

## Final Task

### Subtask:
Conclude the project notes with a summary of the analysis and potential future improvements or next steps.


## Summary:

### Data Analysis Key Findings

*   The project successfully established a framework for analyzing a resume against a job description, providing an introduction to its purpose of identifying relevant skills and calculating a match percentage.
*   Resume processing involved uploading files using `google.colab.files.upload()` and extracting text with a custom `extract_text` function capable of handling both PDF (using `PyPDF2`) and DOCX (using `docx`) formats.
*   The job description was defined as a crucial multi-line string input, serving as the benchmark for skill matching and suitability assessment.
*   Text preprocessing was thoroughly applied to both the resume and job description via a `clean_text` function, which performed lowercasing, punctuation removal, tokenization, stop word removal, and lemmatization to standardize the text.
*   Skill extraction utilized a predefined `skill_list` and an `extract_skills` function to identify unique skills present in both the cleaned resume (`resume_skills`) and the cleaned job description (`job_skills`).
*   Similarity between the resume and job description was calculated using a two-step process:
    *   **TF-IDF Vectorization** transformed cleaned texts into numerical vectors, weighting words by frequency and rarity.
    *   **Cosine Similarity** measured the angular similarity between these vectors, producing a `match_percentage`.
*   A comprehensive "Resume Analysis Report" was outlined to summarize the final output, detailing the overall match score, identified matched skills, and skills missing from the resume, serving as a valuable tool for both job seekers and recruiters.

### Insights or Next Steps

*   The established methodology offers a systematic and quantifiable approach to resume screening, enhancing efficiency for recruiters and providing actionable feedback for job seekers.
*   Future enhancements could include implementing more advanced NLP techniques for implicit skill detection, dynamically generating or expanding the `skill_list` from broader industry data, and integrating this analysis into a user-friendly web interface.


# Task
Describe the `app.py` file, explaining its role as the Streamlit frontend for the resume analysis project, and summarize its key functionalities.

## Describe Streamlit Frontend

### Subtask:
Explain that `app.py` contains the Streamlit application, which serves as the frontend for the resume analysis project, and summarize its key functionalities.


### Describe Streamlit Frontend

`app.py` contains the Python code for a Streamlit web application, serving as the interactive graphical user interface (GUI) or frontend for the entire resume analysis project. This application provides a user-friendly way to interact with the backend analysis logic.

#### User Interaction
Users can easily engage with the application through its intuitive interface:
1.  **Resume Upload**: A `st.file_uploader` widget allows users to upload their resume files, supporting both PDF (`.pdf`) and DOCX (`.docx`) formats.
2.  **Job Description Input**: A `st.text_area` provides a convenient space for users to paste the job description against which their resume will be evaluated.
3.  **Analysis Trigger**: A `st.button` labeled "Analyze Resume" initiates the analysis process once both the resume and job description are provided.

#### Core Functionalities Implemented in `app.py`
The Streamlit application integrates all the core logic developed in the previous steps to deliver a comprehensive resume analysis report:

*   **Text Extraction**: The `extract_text` function (utilizing `PyPDF2` for PDFs and `python-docx` for DOCX files) reads and extracts raw text content from the uploaded resume.
*   **Text Cleaning and Preprocessing**: The `clean_text` function performs essential preprocessing steps on both the extracted resume text and the provided job description. This includes lowercasing, removing punctuation, tokenization, filtering out common stop words, and lemmatizing words to their base forms, ensuring a standardized and clean input for analysis.
*   **Skill Extraction**: The `extract_skills` function identifies and lists relevant technical skills from both the preprocessed resume and job description by comparing them against a predefined list of industry-standard skills.
*   **Similarity Calculation**: The application employs TF-IDF (Term Frequency-Inverse Document Frequency) vectorization to convert the cleaned texts into numerical representations, followed by cosine similarity to compute a quantitative `match_percentage` between the resume and job description.
*   **Report Generation**: Finally, the application displays a clear and concise report directly on the web interface, presenting the calculated match score, a list of `matched_skills` (skills found in both documents), and `missing_skills` (skills required by the job but absent from the resume).

### Describe Streamlit Frontend

`app.py` contains the Python code for a Streamlit web application, serving as the interactive graphical user interface (GUI) or frontend for the entire resume analysis project. This application provides a user-friendly way to interact with the backend analysis logic.

#### User Interaction
Users can easily engage with the application through its intuitive interface:
1.  **Resume Upload**: A `st.file_uploader` widget allows users to upload their resume files, supporting both PDF (`.pdf`) and DOCX (`.docx`) formats.
2.  **Job Description Input**: A `st.text_area` provides a convenient space for users to paste the job description against which their resume will be evaluated.
3.  **Analysis Trigger**: A `st.button` labeled "Analyze Resume" initiates the analysis process once both the resume and job description are provided.

#### Core Functionalities Implemented in `app.py`
The Streamlit application integrates all the core logic developed in the previous steps to deliver a comprehensive resume analysis report:

*   **Text Extraction**: The `extract_text` function (utilizing `PyPDF2` for PDFs and `python-docx` for DOCX files) reads and extracts raw text content from the uploaded resume.
*   **Text Cleaning and Preprocessing**: The `clean_text` function performs essential preprocessing steps on both the extracted resume text and the provided job description. This includes lowercasing, removing punctuation, tokenization, filtering out common stop words, and lemmatizing words to their base forms, ensuring a standardized and clean input for analysis.
*   **Skill Extraction**: The `extract_skills` function identifies and lists relevant technical skills from both the preprocessed resume and job description by comparing them against a predefined list of industry-standard skills.
*   **Similarity Calculation**: The application employs TF-IDF (Term Frequency-Inverse Document Frequency) vectorization to convert the cleaned texts into numerical representations, followed by cosine similarity to compute a quantitative `match_percentage` between the resume and job description.
*   **Report Generation**: Finally, the application displays a clear and concise report directly on the web interface, presenting the calculated match score, a list of `matched_skills` (skills found in both documents), and `missing_skills` (skills required by the job but absent from the resume).

### Describe Streamlit Frontend

`app.py` contains the Python code for a Streamlit web application, serving as the interactive graphical user interface (GUI) or frontend for the entire resume analysis project. This application provides a user-friendly way to interact with the backend analysis logic.

#### User Interaction
Users can easily engage with the application through its intuitive interface:
1.  **Resume Upload**: A `st.file_uploader` widget allows users to upload their resume files, supporting both PDF (`.pdf`) and DOCX (`.docx`) formats.
2.  **Job Description Input**: A `st.text_area` provides a convenient space for users to paste the job description against which their resume will be evaluated.
3.  **Analysis Trigger**: A `st.button` labeled "Analyze Resume" initiates the analysis process once both the resume and job description are provided.

#### Core Functionalities Implemented in `app.py`
The Streamlit application integrates all the core logic developed in the previous steps to deliver a comprehensive resume analysis report:

*   **Text Extraction**: The `extract_text` function (utilizing `PyPDF2` for PDFs and `python-docx` for DOCX files) reads and extracts raw text content from the uploaded resume.
*   **Text Cleaning and Preprocessing**: The `clean_text` function performs essential preprocessing steps on both the extracted resume text and the provided job description. This includes lowercasing, removing punctuation, tokenization, filtering out common stop words, and lemmatizing words to their base forms, ensuring a standardized and clean input for analysis.
*   **Skill Extraction**: The `extract_skills` function identifies and lists relevant technical skills from both the preprocessed resume and job description by comparing them against a predefined list of industry-standard skills.
*   **Similarity Calculation**: The application employs TF-IDF (Term Frequency-Inverse Document Frequency) vectorization to convert the cleaned texts into numerical representations, followed by cosine similarity to compute a quantitative `match_percentage` between the resume and job description.
*   **Report Generation**: Finally, the application displays a clear and concise report directly on the web interface, presenting the calculated match score, a list of `matched_skills` (skills found in both documents), and `missing_skills` (skills required by the job but absent from the resume).
